In [42]:
import pandas as pd
import numpy as np
import re

In [3]:
madden = pd.read_csv("maddenratings.csv")
madden.head()

,Full Name,Team,Position,college,awareness_rating,throwPower_rating,kickReturn_rating,leadBlock_rating,strength_rating,bCVision_rating,...,archetype,blockShedding_rating,runBlockFinesse_rating,teamId,agility_rating,fullNameForSearch,overall_rating,passBlockFinesse_rating,age,primaryKey
0,Aaron Donald,Rams,RE,Pittsburgh,99,20,10,23,99,49,...,DE_PowerRusher,97,45,24,86,Aaron Donald,99,45,31,10852
1,Davante Adams,Raiders,WR,Fresno State,99,45,45,25,63,94,...,WR_DeepThreat,52,12,23,95,Davante Adams,99,17,29,10823
2,Myles Garrett,Browns,RE,Texas AM,98,55,10,17,96,39,...,DE_PowerRusher,90,45,5,85,Myles Garrett,99,45,26,12520
3,Trent Williams,49ers,LT,Oklahoma,99,12,10,98,98,10,...,OT_Power,32,99,15,73,Trent Williams,99,92,34,9800
4,Cooper Kupp,Rams,WR,Eastern Wash.,99,59,84,34,66,96,...,WR_Slot,28,28,24,95,Cooper Kupp,98,34,29,12558


In [4]:
#this function takes in their rating and gives them a class from 1-5 (detailed in the readme)
def rating_class(rating):
    if rating < 76:
        return 1
    if rating >= 76 and rating < 83:
        return 2
    if rating >= 83 and rating < 89:
        return 3
    if rating >= 89 and rating < 95:
        return 4
    if rating >= 95:
        return 5
    
madden["class"] = list(map(rating_class, madden["overall_rating"]))

In [5]:
madden[["Full Name", "overall_rating", "class"]]

,Full Name,overall_rating,class
0,Aaron Donald,99,5
1,Davante Adams,99,5
2,Myles Garrett,99,5
3,Trent Williams,99,5
4,Cooper Kupp,98,5
...,...,...,...
2363,Ross Matiscik,26,1
2364,Zach Triner,26,1
2365,Joe Cardona,25,1
2366,Matthew Orzech,25,1


In [6]:
#this will only get madden players who play skill positions
skill = madden[(madden["Position"]=="WR") | (madden["Position"]=="RB") | (madden["Position"]=="QB") | 
       (madden["Position"]=="TE")]


In [91]:
skill[["Full Name", "overall_rating", "class"]][skill["overall_rating"]>=70].head(5)

,Full Name,overall_rating,class
1,Davante Adams,99,5
4,Cooper Kupp,98,5
6,Travis Kelce,98,5
9,George Kittle,97,5
10,Tom Brady,97,5


In [12]:
passing = pd.read_csv("2021-passing.csv")
passing.head()

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD
0,1,Tom Brady*,TAM,44,QB,17,17,13-4-0,485,719,...,312.7,102.1,68.1,22,144,3.0,6.98,7.41,3.0,5.0
1,2,Justin Herbert*,LAC,23,QB,17,17,9/8/2000,443,672,...,294.9,97.7,65.6,31,214,4.4,6.83,6.95,5.0,5.0
2,3,Matthew Stafford,LAR,33,QB,17,17,12/5/2000,404,601,...,287.4,102.9,63.8,30,243,4.8,7.36,7.45,3.0,4.0
3,4,Patrick Mahomes*,KAN,26,QB,17,17,12/5/2000,436,658,...,284.6,98.5,62.2,28,146,4.1,6.84,7.07,3.0,3.0
4,5,Derek Carr,LVR,30,QB,17,17,10/7/2000,428,626,...,282.6,94.0,52.4,40,241,6.0,6.85,6.60,3.0,6.0


In [16]:
rushing = pd.read_csv("2021-rushing.csv",header=1)
rushing.head()

,Rk,Player,Tm,Age,Pos,G,GS,Att,Yds,TD,1D,Lng,Y/A,Y/G,Fmb
0,1,Jonathan Taylor*+,IND,22,RB,17,17,332,1811,18,107,83,5.5,106.5,4
1,2,Najee Harris*,PIT,23,RB,17,17,307,1200,7,62,37,3.9,70.6,0
2,3,Joe Mixon*,CIN,25,RB,16,16,292,1205,13,60,32,4.1,75.3,2
3,4,Antonio Gibson,WAS,23,RB,16,14,258,1037,7,65,27,4.0,64.8,6
4,5,Dalvin Cook*,MIN,26,RB,13,13,249,1159,6,57,66,4.7,89.2,3


In [33]:
receiving = pd.read_csv("2021-receiving.csv",header=0)
receiving.head()

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Cooper Kupp*+,LAR,28,WR,17,17,191,145,75.90%,1947,13.4,16,89,59,10.2,8.5,114.5,0
1,2,Davante Adams*+,GNB,29,WR,16,16,169,123,72.80%,1553,12.6,11,84,59,9.2,7.7,97.1,0
2,3,Tyreek Hill*,KAN,27,WR,17,16,159,111,69.80%,1239,11.2,9,75,75,7.8,6.5,72.9,2
3,4,Justin Jefferson*,MIN,22,WR,17,17,167,108,64.70%,1616,15.0,10,75,56,9.7,6.4,95.1,1
4,5,Mark Andrews*+,BAL,26,TE,17,9,153,107,69.90%,1361,12.7,9,75,43,8.9,6.3,80.1,1


# Making a Model on just WRs and TEs to figure out functionality

In [51]:
wrte_rating = madden[(madden["Position"]=="WR") | (madden["Position"]=="TE")]
wrte_rating = wrte_rating.rename(columns={"Full Name":"Player"})

#lets filter this to only players whose rating is above 65
wrte_rating = wrte_rating[wrte_rating["overall_rating"] >= 65]
wrte_rating.head()

,Player,Team,Position,college,awareness_rating,throwPower_rating,kickReturn_rating,leadBlock_rating,strength_rating,bCVision_rating,...,blockShedding_rating,runBlockFinesse_rating,teamId,agility_rating,fullNameForSearch,overall_rating,passBlockFinesse_rating,age,primaryKey,class
1,Davante Adams,Raiders,WR,Fresno State,99,45,45,25,63,94,...,52,12,23,95,Davante Adams,99,17,29,10823,5
4,Cooper Kupp,Rams,WR,Eastern Wash.,99,59,84,34,66,96,...,28,28,24,95,Cooper Kupp,98,34,29,12558,5
6,Travis Kelce,Chiefs,TE,Cincinnati,97,70,6,59,79,92,...,40,66,9,86,Travis Kelce,98,63,32,890,5
9,George Kittle,49ers,TE,Iowa,96,30,31,65,82,90,...,31,71,15,85,George Kittle,97,58,28,12836,5
11,Tyreek Hill,Dolphins,WR,West Alabama,90,41,92,20,64,97,...,21,12,12,98,Tyreek Hill,97,19,28,17838,5


In [93]:
wrte_stats = receiving[(receiving["Pos"]=="WR") | (receiving["Pos"]=="TE")]

#lets get rid of the "+" and "*" from their names so we can join the tables
wrte_stats["Player"] = wrte_stats["Player"].map(lambda x: re.sub(r'[*+]', '', x))
wrte_stats["Ctch%"] = wrte_stats["Ctch%"].map(lambda x: re.sub(r'[%]', '', x))
wrte_stats.head()

/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_87192/3569740911.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrte_stats["Player"] = wrte_stats["Player"].map(lambda x: re.sub(r'[*+]', '', x))
/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_87192/3569740911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrte_stats["Ctch%"] = wrte_stats["Ctch%"].map(lambda x: re.sub(r'[%]', '', x))


,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Cooper Kupp,LAR,28,WR,17,17,191,145,75.90,1947,13.4,16,89,59,10.2,8.5,114.5,0
1,2,Davante Adams,GNB,29,WR,16,16,169,123,72.80,1553,12.6,11,84,59,9.2,7.7,97.1,0
2,3,Tyreek Hill,KAN,27,WR,17,16,159,111,69.80,1239,11.2,9,75,75,7.8,6.5,72.9,2
3,4,Justin Jefferson,MIN,22,WR,17,17,167,108,64.70,1616,15.0,10,75,56,9.7,6.4,95.1,1
4,5,Mark Andrews,BAL,26,TE,17,9,153,107,69.90,1361,12.7,9,75,43,8.9,6.3,80.1,1


In [94]:
#now lets join the two dataframes on the names

wrte = wrte_rating[["Player", "overall_rating", "class"]].merge(wrte_stats, how="left", on="Player")

In [95]:
wrte = wrte.dropna(subset=['Yds'])
wrte

,Player,overall_rating,class,Rk,Tm,Age,Pos,G,GS,Tgt,...,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,Davante Adams,99,5,2.0,GNB,29.0,WR,16.0,16.0,169.0,...,72.80,1553.0,12.6,11.0,84.0,59.0,9.2,7.7,97.1,0.0
1,Cooper Kupp,98,5,1.0,LAR,28.0,WR,17.0,17.0,191.0,...,75.90,1947.0,13.4,16.0,89.0,59.0,10.2,8.5,114.5,0.0
2,Travis Kelce,98,5,13.0,KAN,32.0,TE,16.0,16.0,134.0,...,68.70,1125.0,12.2,9.0,63.0,69.0,8.4,5.8,70.3,1.0
3,George Kittle,97,5,36.0,SFO,28.0,TE,14.0,14.0,94.0,...,75.50,910.0,12.8,6.0,42.0,48.0,9.7,5.1,65.0,2.0
4,Tyreek Hill,97,5,3.0,KAN,27.0,WR,17.0,16.0,159.0,...,69.80,1239.0,11.2,9.0,75.0,75.0,7.8,6.5,72.9,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,Eric Saubert,65,1,318.0,DEN,27.0,TE,17.0,4.0,12.0,...,66.70,47.0,5.9,1.0,2.0,11.0,3.9,0.5,2.8,0.0
331,Kylen Granson,65,1,279.0,IND,23.0,TE,17.0,0.0,15.0,...,73.30,106.0,9.6,0.0,4.0,27.0,7.1,0.6,6.2,0.0
332,Noah Gray,65,1,325.0,KAN,22.0,TE,16.0,1.0,10.0,...,70.00,36.0,5.1,1.0,2.0,8.0,3.6,0.4,2.3,0.0
334,Richard Rodgers,65,1,449.0,PHI,29.0,TE,3.0,0.0,2.0,...,100.00,11.0,5.5,0.0,0.0,9.0,5.5,0.7,3.7,0.0


In [96]:
wrte.columns

Index(['Player', 'overall_rating', 'class', 'Rk', 'Tm', 'Age', 'Pos', 'G',
       'GS', 'Tgt', 'Rec', 'Ctch%', 'Yds', 'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt',
       'R/G', 'Y/G', 'Fmb'],
      dtype='object')

# Creating a WRTE model

In [105]:
# first lets set up our X and y dataframes
X = wrte[["Tgt", "Ctch%", "Yds", "Y/R", "TD", "Y/G"]]
y = wrte[["class"]]

In [106]:
X.head()

,Tgt,Ctch%,Yds,Y/R,TD,Y/G
0,169.0,72.80,1553.0,12.6,11.0,97.1
1,191.0,75.90,1947.0,13.4,16.0,114.5
2,134.0,68.70,1125.0,12.2,9.0,70.3
3,94.0,75.50,910.0,12.8,6.0,65.0
4,159.0,69.80,1239.0,11.2,9.0,72.9


In [119]:
y = np.ravel(y)
y

array([5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [120]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metri
import lightgbm
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.model_selection import cross_val_score, cross_val_predict


def objective(trial):
    classifier_name = trial.suggest_categorical("classifier", ["GBM"])
    if classifier_name == "GBM":
        #p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}
        svc_c = trial.suggest_float("learning_rate", .001, .15, log=True)
        svc_c1 = trial.suggest_int("n_estimators", 5, 50, log=True)
        svc_c2= trial.suggest_int("max_depth", 2, 30, log=True)
        #model = GradientBoostingClassifier()
        classifier_obj = GradientBoostingClassifier(random_state=0, learning_rate = svc_c, max_depth=svc_c2, n_estimators=svc_c1)
    cv = LeaveOneOut()
    y_pred = cross_val_predict(classifier_obj, X, y, cv=cv)
    accuracy = np.diag(pd.crosstab(y_pred,y)).sum()/len(y)
    return accuracy


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2023-05-04 17:21:17,244] A new study created in memory with name: no-name-084eb5c3-ae1a-459e-8a71-e129cbb9f6ab
[I 2023-05-04 17:21:39,813] Trial 0 finished with value: 0.5630630630630631 and parameters: {'classifier': 'GBM', 'learning_rate': 0.002169144003366332, 'n_estimators': 44, 'max_depth': 2}. Best is trial 0 with value: 0.5630630630630631.
[I 2023-05-04 17:21:49,301] Trial 1 finished with value: 0.5630630630630631 and parameters: {'classifier': 'GBM', 'learning_rate': 0.03152519215906972, 'n_estimators': 5, 'max_depth': 25}. Best is trial 0 with value: 0.5630630630630631.
